# Determine Point Totals and Success of Optimal Rosters based on Quantile Predictions

## read in quantile predictions

In [94]:
using JuMP, Gurobi, CSV, DataFrames

In [176]:
all_data = DataFrame(CSV.File("period_cum_season.csv"))
;

## read in point total required to make money in a given week (https://rotogrinders.com/resultsdb)

In [177]:
money_mat = [
    2017 9 141
    2017 10 115
    2017 11 135
    2017 12 139
    2017 13 126
    2017 14 139
    2017 15 132
    2017 16 133
    2018 9 152
    2018 10 130
    2018 11 135
    2018 12 159
    2018 13 149
    2018 14 141
    2018 15 125
    2018 16 163
    2019 9 156
    2019 10 147
    2019 11 136
    2019 12 126
    2019 13 140
    2019 14 141
    2019 15 156
    2019 16 166
]
;

In [178]:
#all columns that are not various weights of quantile predictions produced in create_quantile_predictions.R
not_predictions = [:Column1,:Week,:Year,:GID,:Name,:Pos_Def,:Pos_QB,:Pos_RB,
    :Pos_TE,:Pos_WR,:Team,:Oppt,:DKP,:DKSalary,:salary_group,:week_rank]
#various weights of quantile predictions being tested. Created in create_quantile_predictions.R
predictions = [name for name in names(all_data) if name ∉ not_predictions]
;

In [182]:
extras = 9 # number of extra suboptimal rosters to create
results = Array{Any}(undef,length(predictions)*length(2017:2019)*length(9:16),5+extras) # to store results
row = 1 #used to iterate over when populating "results" array
eps = 1 #degree to which objective value must decrease when finding suboptimal roster. 

#perform analysis in each week-year combination for second half of 2017-2019.
for year in 2017:2019
    for week in 9:16
        println(year,"-",week)
        #subset to week being analyzed for both predictions and money line
        df = all_data[(all_data[:,:Week] .== week) .& (all_data[:,:Year] .== year),:]
        money = money_mat[(money_mat[:,1] .== year) .& (money_mat[:,2] .== week),3]

        #get optimal roster for each combination of quantile weights
        for prediction in predictions
            #optimization model
            model = Model(Gurobi.Optimizer)
            set_optimizer_attributes(model, "OutputFlag"=>0)

            num_players = size(df)[1]
            # Insert variables and constraints
            @variable(model,z[i=1:num_players],Bin)
            @constraint(model,sum(z[i]*df[i,:DKSalary] for i=1:num_players) <= 50000)
            @constraint(model,sum(z[i]*df[i,:Pos_Def] for i=1:num_players) == 1)
            @constraint(model,sum(z[i]*df[i,:Pos_QB] for i=1:num_players) == 1)
            @constraint(model,sum(z[i]*df[i,:Pos_TE] for i=1:num_players) >= 1)
            @constraint(model,sum(z[i]*df[i,:Pos_RB] for i=1:num_players) >= 2)
            @constraint(model,sum(z[i]*df[i,:Pos_WR] for i=1:num_players) >= 3)
            @constraint(model,sum(z[i]*(df[i,:Pos_TE] + df[i,:Pos_RB] + df[i,:Pos_WR]) for i=1:num_players) == 7)
            @objective(model,Max, sum(z[i]*df[i,prediction] for i=1:num_players))

            # Optimize
            optimize!(model)
            obj = objective_value(model)
            #actual points from optimal roster (chosen based on predicted points)
            points = sum(df[value.(z) .== 1,:DKP])
            results[row,1] = year
            results[row,2] = week
            results[row,3] = prediction
            results[row,4] = money[1]
            results[row,5] = points
            
            #create additional suboptimal rosters
            for i in 1:extras
                @constraint(model,sum(z[i]*df[i,prediction] for i=1:num_players) <= obj - eps)
                optimize!(model)
                obj = objective_value(model)
                #actual points from optimal roster (chosen based on predicted points)
                points = sum(df[value.(z) .== 1,:DKP])
                results[row,i+5] = points
            end
            row = row + 1
        end
    end
end

2017-9
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academ

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic lice

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic lice

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic lice

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic lice

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic lice

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic lice

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic lice

## save results for further analysis in R

In [183]:
CSV.write("output/optimization_results.csv",DataFrame(results))
;